In [ ]:
#telnet smtp.interia.pl 587 ----- TAK NIE ZADZIALA PRZEZ STARRTTLS 
openssl s_client -starttls smtp -connect smtp.interia.pl:587
EHLO interia.pl
AUTH LOGIN
cGFzMjAxN0BpbmVyaWEucGw=
UDRTSW5mMjAxNw== 
MAIL FROM:<pas2017@interia.pl>
RCPT TO:<pasinf2017@interia.pl>
DATA
Subject: dasdonasod

testowa wiadomość foiasjfoiaf.
.
QUIT


openssl s_client -starttls smtp -connect smtp.interia.pl:587
EHLO example.com
AUTH LOGIN
cGFzMjAxN0BpbmVyaWEucGw=
UDRTSW5mMjAxNw== 
MAIL FROM:<pas2017@interia.pl>
RCPT TO:<pasinf2017@interia.pl>
RCPT TO:<testintinteria@example.com>
DATA
Subject: Test

2 ODNIORCOW TERAZ.
.
QUIT


openssl s_client -starttls smtp -connect smtp.interia.pl:587
EHLO example.com
AUTH LOGIN
cGFzMjAxN0BpbmVyaWEucGw=
UDRTSW5mMjAxNw== 
MAIL FROM:<spoofmail@interia.pl>
RCPT TO:<pasinf2017@interia.pl>
DATA
Subject: Test

boooospoof mail.
.
QUIT


openssl s_client -starttls smtp -connect smtp.interia.pl:587
EHLO example.com
AUTH LOGIN
cGFzMjAxN0BpbmVyaWEucGw=
UDRTSW5mMjAxNw==
MAIL FROM:<pas2017@interia.pl>
RCPT TO:<pasinf2017@interia.pl>
DATA
Subject: MIME
From: pas2017@interia.pl
To: pasinf2017@interia.pl
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary="XYZ"

--XYZ
Content-Type: text/plain

To jest zwykła wiadomość.

--XYZ
Content-Type: text/plain; name="plik.txt"
Content-Disposition: attachment; filename="plik.txt"
Content-Transfer-Encoding: base64

[plik.txt.b64]

--XYZ--
.
QUIT



openssl s_client -starttls smtp -crlf -connect smtp.interia.pl:587
EHLO example.com
AUTH LOGIN
cGFzMjAxN0BpbmVyaWEucGw=
UDRTSW5mMjAxNw==
MAIL FROM:<pas2017@interia.pl>
RCPT TO:<pasinf2017@interia.pl>
DATA
From: pas2017@interia.pl
To: pasinf2017@interia.pl
Subject: testing2
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary="boundary123"

--boundary123
Content-Type: text/plain; charset="UTF-8"

prezez openssl

--boundary123
Content-Type: image/jpeg; name="obrazek.jpg"
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="obrazek.jpg"

[obrazek.b64]

--boundary123--
.


SyntaxError: invalid syntax (1996782755.py, line 1)

In [ ]:
import smtplib
from email.message import EmailMessage


smtp_server = "smtp.interia.pl"
smtp_port = 587


sender_email = input("Podaj swoj mail:")
password = input("podaj swoje hasło: ")
recipient_email = input("Podaj adres odbiorcy (lub kilka, rozdzielone przecinkami): ")
subject = input("podaj temat wiadomości: ")
body = input("podaj wiadomosc: ")

msg = EmailMessage()
msg["From"] = sender_email
msg["To"] = recipient_email
msg["Subject"] = subject
msg.set_content(body)

try:
    with smtplib.SMTP(smtp_server, smtp_port, timeout=5) as server:
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(sender_email, password)
        server.send_message(msg)
        print("wyslano!")
except Exception as e:
    print(f"Blad:  {e}")



Błąd podczas wysyłania wiadomości: timed out


In [ ]:
import socket
import ssl
import base64

def txtToBase64(filepath):
    with open(filepath, 'rb') as f:
        return base64.b64encode(f.read()).decode()

def base64_str(s):
    return base64.b64encode(s.encode()).decode()

def recv(sock):
    data = sock.recv(4096).decode()
    print(data)
    return data

def send(sock, cmd):
    sock.send((cmd + '\r\n').encode())


host = "smtp.interia.pl"
port = 587

sender_email = input("Podaj swoj mail:")
password = input("podaj swoje hasło: ")
recipient_email = input("Podaj adres odbiorcy:")
subject = input("podaj temat wiadomości: ")
body = input("podaj wiadomosc: ")
filename = "plik.txt"
encoded_file = txtToBase64(filename)

sock = socket.create_connection((host, port))
recv(sock)
send(sock, "EHLO localhost")
recv(sock)

send(sock, "STARTTLS")
recv(sock)

context = ssl.create_default_context()
sock = context.wrap_socket(sock, server_hostname=host)

send(sock, "EHLO localhost")
recv(sock)

send(sock, "AUTH LOGIN")
recv(sock)
send(sock, base64_str(sender))
recv(sock)
send(sock, base64_str(password))
recv(sock)

send(sock, f"MAIL FROM:<{sender}>")
recv(sock)
send(sock, f"RCPT TO:<{recipient}>")
recv(sock)
send(sock, "DATA")
recv(sock)

boundary = "boundary123"

message = f"""From: {sender}
To: {recipient}
Subject: {subject}
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary={boundary}

--{boundary}
Content-Type: text/plain; charset=utf-8

{body}

--{boundary}
Content-Type: text/plain; name="{filename}"
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="{filename}"

{encoded_file}
--{boundary}--
.
"""

send(sock, message)
recv(sock)

send(sock, "QUIT")
recv(sock)
sock.close()
print("Wiadomość wysłana.")


In [ ]:
import socket
import ssl
import base64

def imageToBase64(filepath):
    with open(filepath, 'rb') as f:
        return base64.b64encode(f.read()).decode()


sender_email = input("Podaj swoj mail:")
password = input("podaj swoje hasło: ")
recipient_email = input("Podaj adres odbiorcy:")
subject = input("podaj temat wiadomości: ")
body = input("podaj wiadomosc: ")
image_path = input("Podaj path do obrazka: ")

encoded_image = imageToBase64(image_path)
filename = image_path.split('/')[-1] #nzawaz pliku

sock = socket.create_connection((host, port))
recv(sock)
send(sock, "EHLO localhost")
recv(sock)

send(sock, "STARTTLS")
recv(sock)

context = ssl.create_default_context()
sock = context.wrap_socket(sock, server_hostname=host)

send(sock, "EHLO localhost")
recv(sock)

send(sock, "AUTH LOGIN")
recv(sock)
send(sock, strToBase64(sender))
recv(sock)
send(sock, strToBase64(password))
recv(sock)

send(sock, f"MAIL FROM:<{sender}>")
recv(sock)
send(sock, f"RCPT TO:<{recipient}>")
recv(sock)
send(sock, "DATA")
recv(sock)

boundary = "boundary123"

message = f"""From: {sender}
To: {recipient}
Subject: {subject}
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary={boundary}

--{boundary}
Content-Type: text/plain; charset=utf-8

{body}

--{boundary}
Content-Type: image/jpeg; name="{filename}"
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="{filename}"

{encoded_image}
--{boundary}--
.
"""

send(sock, message)
recv(sock)

send(sock, "QUIT")
recv(sock)
sock.close()

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

smtp_server = "smtp.interia.pl"  
smtp_port = 587
username = "pas2017@interia.pl"

from_addr = username
to_addr = "pasinf2017@interia.pl"

msg = MIMEMultipart("alternative")
msg["Subject"] = "html message"
msg["From"] = from_addr
msg["To"] = to_addr


html_text = """
<html>
  <body>
    <p> test t <b>grubego</b> tekstu,<br>
       <i>pochylony</i> tekst,<br>
       <u>podkkreslony</u> tekst,<br>
        <span style="color:red;">niebieski</span> tekst.
    </p>
  </body>
</html>
"""

html_part = MIMEText(html_text, "html")
msg.attach(html_part)

try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.ehlo()                
        server.starttls()           
        server.ehlo()               
        server.login(username, "P4SInf2017")
        server.sendmail(from_addr, to_addr, msg.as_string())
        print("wysłano pomyślnie.")
except Exception as e:
    print(f"error: {e}")

In [ ]:
import socket

HOST = '127.0.0.1'
PORT = 2525  

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

conn, addr = server_socket.accept()

conn.send(b"welcome to SMTP server\r\n")

while True:
    data = conn.recv(1024).decode().strip()
    if not data:
        break

    print(f"< {data}")

    cmd = data.upper()

    if cmd.startswith("EHLO") or cmd.startswith("HELO"):
        conn.send(b"250 Hello localhost\r\n")

    elif cmd.startswith("MAIL FROM"):
        conn.send(b"250 OK\r\n")

    elif cmd.startswith("RCPT TO"):
        conn.send(b"250 OK\r\n")

    elif cmd == "DATA":
        conn.send(b"354 End with <CR><LF>.<CR><LF>\r\n")
        # Odbieramy dane wiadomości do momentu linii z kropką
        message_lines = []
        while True:
            line = conn.recv(1024).decode().strip()
            if line == ".":
                break
            message_lines.append(line)
        print("Wiadomość została zasymulowana.")
        conn.send(b"250 Message accepted for delivery\r\n")

    elif cmd == "QUIT":
        conn.send(b"221 Bye\r\n")
        break

    else:
        conn.send(b"502 Command not implemented\r\n")

conn.close()
server_socket.close()
print("Serwer zakończył działanie.")